In [1]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.environments.gym_wrapper import GymWrapper

from utils import make_env, ConvertTypeObservation


num_stack = 4
# skipframe_div = 10

env = make_env(num_stack=num_stack, truncate_on_shiphit=True)
env = ConvertTypeObservation(env)
env = GymWrapper(env)
tf_env = TFPyEnvironment(env)

In [2]:
import tensorflow as tf

from matplotlib import pyplot as plt


obs = tf_env.reset()

# action = tf_env.action_space.sample()
obs = tf_env.step(3)[3]

plt.figure(figsize=(8, 5))
for i in range(1, num_stack+1):
    plt.subplot(2, 2, i)
    plt.imshow(tf.cast(obs[0, ..., i-1], "uint8"), cmap="gray")

In [3]:
from tensorflow.keras import layers

from tf_agents.networks.actor_distribution_network import ActorDistributionNetwork
from tf_agents.networks.value_network import ValueNetwork


preprocessing_layer = layers.Lambda(lambda obs: tf.cast(obs, tf.float32) / 255.)
conv_layer_params = [(32, (8, 8), 4), (64, (4, 4), 2), (64, (3, 3), 2), (128, (3, 3), 2), (128, (3, 3), 1)]
fc_layer_params=[512]

actor_net = ActorDistributionNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params
)

value_net = ValueNetwork(
    tf_env.observation_spec(),
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params
)

In [4]:
train_iters = 50_000
update_period = 20
buffer_size = 65_000

num_epochs = 12
batch_size = 64
lr = 3e-5

In [5]:
from tensorflow.keras import optimizers

from tf_agents.agents.ppo.ppo_agent import PPOAgent


train_step = tf.Variable(0)

optimizer = optimizers.Adam(learning_rate=lr)

agent = PPOAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    optimizer=optimizer,
    actor_net=actor_net,
    value_net=value_net,
    num_epochs=num_epochs,
    train_step_counter=train_step,
    normalize_observations=True,
    normalize_rewards=True
)

agent.initialize()

In [6]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer


replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=buffer_size
)
replay_buffer_observer = replay_buffer.add_batch

dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2
).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [7]:
from tf_agents.metrics import tf_metrics
from tf_agents.eval.metric_utils import log_metrics

import logging


class ShowProgress:

    def __init__(self, total):
        self.counter = 0
        self.total = total
        
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")
            

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
]

logging.getLogger().setLevel(logging.INFO)
log_metrics(train_metrics)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 0
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


In [8]:
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver


collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + train_metrics,
    num_steps=update_period
)

init_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch, ShowProgress(buffer_size//4)],
    num_steps=buffer_size//4
)

final_time_step, final_policy_state = init_driver.run()

16200/16250

In [9]:
from tf_agents.utils.common import function


collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [10]:
def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(tf_env.batch_size)

    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        
        train_loss = agent.train(trajectories)

        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")

        if iteration % (n_iterations//50) == 0:
            log_metrics(train_metrics)

    log_metrics(train_metrics)
    tf.saved_model.save(agent._actor_net, f"models/Alien Invasion ppo ({n_iterations} iters)/actor net TF-Agents")
    tf.saved_model.save(agent._value_net, f"models/Alien Invasion ppo ({n_iterations} iters)/value net TF-Agents")

In [11]:
train_agent(n_iterations=train_iters)

INFO:absl: 
		 NumberOfEpisodes = 1
		 EnvironmentSteps = 20
		 AverageReturn = -601.0599975585938
		 AverageEpisodeLength = 13.0


999 loss:-0.18425

INFO:absl: 
		 NumberOfEpisodes = 12
		 EnvironmentSteps = 20020
		 AverageReturn = -565.8275756835938
		 AverageEpisodeLength = 1675.4000244140625


1999 loss:-0.21356

INFO:absl: 
		 NumberOfEpisodes = 24
		 EnvironmentSteps = 40020
		 AverageReturn = -566.9017944335938
		 AverageEpisodeLength = 1677.5999755859375


...

36999 loss:-0.07688

INFO:absl: 
		 NumberOfEpisodes = 412
		 EnvironmentSteps = 740020
		 AverageReturn = -510.1216735839844
		 AverageEpisodeLength = 1803.0999755859375


37999 loss:0.040525

INFO:absl: 
		 NumberOfEpisodes = 423
		 EnvironmentSteps = 760020
		 AverageReturn = -502.53961181640625
		 AverageEpisodeLength = 1804.800048828125


38141 loss:-0.06924

In [ ]:
frames = []

tf_env.pyenv.envs[0].reset()

def save_frames(trajectory):
    global frames
    frames.append(tf_env.pyenv.envs[0].render())

watch_driver = DynamicStepDriver(
    tf_env,
    agent.policy,
    observers=[save_frames, ShowProgress(10_000)],
    num_steps=10_000)
final_time_step, final_policy_state = watch_driver.run()

In [ ]:
import PIL

import os


image_path = os.path.join("rl videos", f"alieninvasion_ppo_tfagents_{train_iters}.gif")

frame_images = []
for frame in frames:
    frame.shape = frame.shape[:-1]
    frame_images.append(PIL.Image.fromarray(frame))

frame_images[0].save(image_path, format='GIF',
                     append_images=frame_images[1:],
                     save_all=True,
                     duration=30,
                     loop=0)